In [1]:
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.entity_list import EntityList
from time import time, sleep

Loaded constant2epsilon cache of size: (300000,)


In [2]:
t0 = time()
# df = pd.read_parquet("/home/ruchi/1B_rows_dataset_sample.parquet")
df = pd.read_parquet("1M_rows_dataset_sample.parquet")
tf = time() - t0

print(f"Time taken to read file with {df.shape[0]/1e6} million rows : {tf} seconds")

Time taken to read file with 1.0 million rows : 0.22391700744628906 seconds


In [3]:
df.head()

,tweet_id,impressions,tweet_date_time,date,time,user_id,url,publication_title,ad_fontes_bias,ad_fontes_reliability,domain
0,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,92256,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
1,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,418264,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
2,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,195053,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
3,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,590420,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
4,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,602475,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com


In [4]:
df['impressions']

0         9029
1         9029
2         9029
3         9029
4         9029
          ... 
999995    9709
999996    9709
999997    9709
999998    9709
999999    9709
Name: impressions, Length: 1000000, dtype: int64

In [5]:
scale = 1 #10_000  # This is put here to stop int32 overflow errors

t0 = time()
impressions = df['impressions'].to_numpy(np.int64) // scale
data_subjects = EntityList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [ ]:
df['impressions'].min()

In [ ]:
f = lambda sigma: np.square(726)/(2*(sigma**2))

In [ ]:
f(1e5)

In [ ]:
t0 = time()
# tweets_data = sy.Tensor(impressions).private(min_val=750_000/scale, max_val=20e6/scale, entities = data_subjects,ndept=True)  # RUn this for 1 billion rows
tweets_data = sy.Tensor(impressions).private(min_val=500/scale, max_val=25000/scale, entities = data_subjects,ndept=True)  # Run this for 1 million rows
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

In [ ]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)

In [ ]:
%%time
name = f"Tweets data- {time()}"

domain_node.load_dataset(
    assets={name: tweets_data},
    name=name,
    description=" Tweets- 1B rows",
    use_blob_storage=True
)

In [ ]:
domain_node.datasets

In [ ]:
data = domain_node.datasets[-1][name]

In [ ]:
%%time
sum_result = data.sum()
sum_result.block

In [ ]:
sum_result.exists

In [ ]:
published_result = sum_result.publish(sigma=1e5)

Sigma, PB_spend
- 100, 859.4086891067999
- 1e4, 7.766216625311721
- 1e5, 7.766216625311721
- 2e5, 7.766216625311721
- 5e5, 7.766216625311721
- 1e7, 7.766216625311721
- 1e9, 7.766216625311721

Is PB not being calculated correctly?

In [ ]:
while not published_result.exists:
    sleep(1)

In [ ]:
published_result

In [ ]:
published_result.request()

In [ ]:
domain_node.requests

In [ ]:
domain_node.requests[-1].accept()

In [ ]:
domain_node.requests

In [ ]:
result_of_our_hard_labour = published_result.get()

In [ ]:
result_of_our_hard_labour

In [ ]:
impressions.sum()

In [ ]:
def percentage_error(true_value, noisy_value):
    return (true_value - noisy_value)/true_value * 100

In [ ]:
percentage_error(true_value=impressions.sum(), noisy_value=result_of_our_hard_labour)

In [ ]:
(1644527104 - 11941995258)/11941995258 * 100

In [ ]:
11941995258 /1e6

In [ ]:
domain_node.privacy_budget